<a href="https://colab.research.google.com/github/DorShabat/Cryptology-Project/blob/main/ECDH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
%%capture
!pip install cryptography
!pip install cryptography.hazmat.primitives.kdf.hkdf

# The Elliptic Curve Diffie-Hellman (ECDH)

## Define the Elliptic Curve

y^2 = x^3 + ax + b mod p


G refers to a specific point on the elliptic curve known as the generator point.

In [46]:
P = 6277101735386680763835789423207666416083908700390324961279
a = 6277101735386680763835789423207666416083908700390324961276
b = 2455155546008943817740293915197451784769108058161191238065
Gx = 602046282375688656758213480587526111916698976636884684818
Gy = 174050332293622031404857552280219410364023488927386650641
G = (Gx, Gy)

In [3]:
def multiply_point_by_scalar_mod_P(point, scalar, P, a):
    """
    Multiplies a point on an elliptic curve by a scalar using the double-and-add algorithm.

    Parameters:
    point (tuple): The point on the elliptic curve (x, y).
    scalar (int): The scalar to multiply the point by.
    P (int): The prime order of the finite field.
    a (int): The coefficient 'a' in the elliptic curve equation y^2 = x^3 + ax + b.

    Returns:
    tuple: The resulting point after multiplication.
    """
    def point_addition(P1, P2, P, a):
        if P1 == (None, None):
            return P2
        if P2 == (None, None):
            return P1
        if P1 == P2:
            return point_doubling(P1, P, a)

        x1, y1 = P1
        x2, y2 = P2

        if x1 == x2 and (y1 + y2) % P == 0:
            return (None, None)

        m = (y2 - y1) * pow(x2 - x1, -1, P) % P
        x3 = (m * m - x1 - x2) % P
        y3 = (m * (x1 - x3) - y1) % P

        return (x3, y3)

    def point_doubling(P1, P, a):
        if P1 == (None, None):
            return (None, None)

        x1, y1 = P1

        m = (3 * x1 * x1 + a) * pow(2 * y1, -1, P) % P
        x3 = (m * m - 2 * x1) % P
        y3 = (m * (x1 - x3) - y1) % P

        return (x3, y3)

    result = (None, None)
    addend = point

    while scalar:
        if scalar & 1:
            result = point_addition(result, addend, P, a)
        addend = point_doubling(addend, P, a)
        scalar >>= 1

    return result

In [39]:
# lets choose:
a = 2
b = 3
P = 97 #prime number

# We'll pick a generator point G on the curve. For this curve, let's use:
G = (3, 6)

## Alice

In [40]:
# private key:
private_key_a = 6
# public key:
A = multiply_point_by_scalar_mod_P(point=G, scalar=private_key_a, P=P, a=a)
print(A)

(3, 6)


## Bob

In [41]:
# private key : b
private_key_b = 7
# public key : B = bG
B = multiply_point_by_scalar_mod_P(point=G, scalar=private_key_b, P=P, a=a)
print(B)

(80, 10)


## extchanging public keys...

`S_A = private_key_a * B`

`S_B = private_key_b * A`

Shared secret: `S_A == S_b`

## Alice

In [42]:
S_A = multiply_point_by_scalar_mod_P(point=B, scalar=private_key_a, P=P, a=a)
print(S_A)

(80, 10)


## Bob

In [43]:
S_B = multiply_point_by_scalar_mod_P(point=A, scalar=private_key_b, P=P, a=a)
print(S_B)

(80, 10)


In [44]:
print(S_A == S_B)

True


In [45]:
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend

# shared secret (an integer)
shared_secret_int = S_A[0]

# Calculate the number of bytes needed to represent the integer
num_bytes = (shared_secret_int.bit_length() + 7) // 8

# Convert the integer to a byte array
shared_secret_bytes = shared_secret_int.to_bytes(num_bytes, byteorder='big')

# Derive a key using HKDF
# Choose the desired output length of the key (e.g., 32 bytes for AES-256)
output_key_length = 16

# Create HKDF instance
hkdf = HKDF(
    algorithm=hashes.SHA256(),
    length=output_key_length,
    salt=None,  # Can be None or a randomly generated value
    info=b'handshake data',  # Optional context-specific info
    backend=default_backend()
)

# Perform key derivation
derived_key = hkdf.derive(shared_secret_bytes)

print("Derived Key:", derived_key.hex())


Derived Key: 53a5f6b499e96a7e0513e099b0bdccb4
